In [ ]:
%pwd

'/app'

In [ ]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip -q install tensorflow   # currently 2.13.1
#!pip install -U transformers

In [ ]:
# export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH     # need to add this to Dockerfile or execute in container to add CUDA libraries to the path in order to detect CUDA libraries
%ls /usr/local/cuda/lib64/libcudart.so
%ls /usr/local/cuda/lib64/libcudart.so.11.0
%ls /usr/local/cuda/lib64/libcudart.so.12.0       # this was not installed!
# %ls /usr/local/cuda/lib64/      # list of all CUDA libraries installed
# export LD_LIBRARY_PATH=/usr/local/lib/python3.8/dist-packages/nvidia/cuda_runtime/lib:$LD_LIBRARY_PATH
# Update Dockerfile for CUDA environment variable, and apt-get update, upgrade, and then install -y cuda-libraries-11-6 cuda-libraries-dev-11-6 cuda-tools-11-6


/usr/local/cuda/lib64/libcudart.so@
ls: cannot access '/usr/local/cuda/lib64/libcudart.so.11.0': No such file or directory
ls: cannot access '/usr/local/cuda/lib64/libcudart.so.12.0': No such file or directory


In [ ]:
#find / -name "libcudart.so*"     # running this in container to check conflicting versions of CUDA libraries gave the below result
  #/usr/local/lib/python3.8/dist-packages/nvidia/cuda_runtime/lib/libcudart.so.12
  #/usr/local/cuda-11.6/targets/x86_64-linux/lib/libcudart.so.11.6.55
  #/usr/local/cuda-11.6/targets/x86_64-linux/lib/libcudart.so.11.0
  #/usr/local/cuda-11.6/targets/x86_64-linux/lib/libcudart.so

In [ ]:
!nvidia-smi

Sat Sep 21 17:20:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.01              Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti     On  | 00000000:01:00.0 Off |                  N/A |
|  0%   57C    P2              68W / 285W |  11944MiB / 12282MiB |     48%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Mistral 7B Instruct



In [ ]:
# Import modules
import torch
import transformers
import setuptools
print(setuptools.__version__)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import time
import tensorrt as trt

74.1.2


2024-09-21 14:34:14.627838: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-21 14:34:14.745956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 14:34:14.791906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 14:34:14.805409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 14:34:14.888424: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
# torch.set_default_device('cuda')      # works on fresh container state
torch.cuda.set_device(0)                # works after installing torchvision and torchaudio since torch version is updated

In [ ]:
# Log into HuggingFace
!huggingface-cli login

In [ ]:
# Enable verbose logging to see progress bar for downloading model from Hugging Face
#from transformers.utils import logging
#logging.set_verbosity_info()
#logging.set_verbosity_error()   # only display error

In [ ]:
torch.cuda.empty_cache()
# !nvidia-smi
print("CUDA availability:",torch.cuda.is_available())
print("Number of devices:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0))

CUDA availability: True
Number of devices: 1
Device name: NVIDIA GeForce RTX 4070 Ti


In [ ]:
#model.to(device)
#device = torch.device("cuda")
#!nvcc --version
#!pip install bitsandbytes==0.43.0
#!wget https://files.pythonhosted.org/packages/72/e5/06ed351cdf8d1d5bf7eb86729e4e3669c6844654354aef3b1bc9da66d0bb/bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl   # bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl
#!pip install bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl

import platform
print(platform.system())   # check system - Linux
!uname -m                  # check architecture - x86_64
!python3 --version
!python3.12 -m pip show bitsandbytes
#!pip uninstall bitsandbytes -y
#!pip install git+https://github.com/TimDettmers/bitsandbytes.git
#!pip list

Linux
x86_64
Python 3.12.5
Name: bitsandbytes
Version: 0.43.3
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, torch
Required-by: 


In [ ]:
start_time = time.time()
"""
from accelerate import init_empty_weights, infer_auto_device_map

# Initialize the model with no weights
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")      ##### Might need to use the sharded version of the model for CPU offloading https://huggingface.co/google/flan-ul2/discussions/8



# Create the device map to offload some layers to the CPU
# Refer to https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
device_map = infer_auto_device_map(model, max_memory={"cpu": "48GB", 0: "12GB"})     # i7-12700k has 128 GB memory; give the system some buffer room to avoid CUDA memory error by assigning 11.5 instead of 12

# Load the model with the device map
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             load_in_8bit=True,
                                             device_map=device_map,
                                             torch_dtype="auto")
"""

# Load the model. Optimize via gradient checkpointing, 8-bit quantization, and automatic device mapping
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             #load_in_8bit=True,   # 8-bit quantization - Quantizing the model can significantly reduce its memory footprint - leverage BitsAndBytes for 8-bit or 4-bit quantization
                                                                  # Loads the model in 8-bit precision, reducing memory footprint by ~1/2 compared to full precision (16-bit or 32-bit)
                                                                  # load_in_4bit=True reduces it further, but comes with slight trade-off in model performance
                                             device_map="cuda:0",   #"cuda:0"  # Automatic device mapping - automatically splits the model layers across available devices (e.g., GPU and CPU) depending on available memory
                                             torch_dtype="auto")


# Gradient checkpointing - reduces memory usage during the forward pass by recomputing certain parts of the graph - i.e., activations - in the backward pass instead of storing intermediate states
model.gradient_checkpointing_enable()

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype="auto")

# Measure loading time
print("===== %s seconds =====" % (time.time() - start_time))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

===== 16.27105689048767 seconds =====


In [ ]:
start_time = time.time()

# Move model to GPU for accelerated inference
model = model.to('cuda')     # to.() not supported for 8-bit. Just use model as is, since correct device has already been assigned to it

# Encode prompt
text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs = encodeds.to('cuda')

# Generate output using the GPU (CUDA accelerated)
ouputs = model.generate(**inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(ouputs)
print(decoded[0])

print("--- %s seconds ---" % (time.time() - start_time))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a simple recipe for homemade mayonnaise that you can whip up in no time at all:

Ingredients:

* 1 cup egg yolks (room temperature)
* 2 tablespoons Dijon mustard
* 2 tablespoons apple cider vinegar
* 1 teaspoon salt
* 1/2 teaspoon black pepper
* 2 cups vegetable oil or combination of vegetable and olive oil

Instructions:

1. In a blender or food processor, combine the egg yolks, Dijon mustard, apple cider vinegar, salt, and pepper. Blend until the mixture is well combined and light in colour.
2. slowly add the vegetable oil to the mixture while the blender or food processor is running. Make sure to pour the oil in a steady stream so that the oil and egg mixture emulsify properly.
3. Blend for a few more minutes unti

In [ ]:
start_time = time.time()

# Practice prompt
text = """<s>[INST] What do you think are the most important things in life? [/INST]
          I think building healthy, long-lasting relationships, a fulfilling career and exciting hobbies are vital components for a happy life!</s>
          [INST] Do you have any life advice to give someone in their early thirties seeking a career transition to machine learning, particularly in LLM's? [/INST]"""

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs = encodeds.to('cuda')
#model.to(device)     # to.() not supported for 8-bit. Just use model as is, since correct device has already been assigned to it

ouputs = model.generate(**inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(ouputs)
print(decoded[0])

print("--- %s seconds ---" % (time.time() - start_time))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] What do you think are the most important things in life? [/INST]
          I think building healthy, long-lasting relationships, a fulfilling career and exciting hobbies are vital components for a happy life!</s>
          [INST] Do you have any life advice to give someone in their early thirties seeking a career transition to machine learning, particularly in LLM's? [/INST] If you have a background in a field that's applicable to machine learning, such as computer science, mathematics or physics, your background knowledge will be useful when transitioning to machine learning. However, your first few years in this field will likely involve learning new skills and acquiring new knowledge. It's important to be open-minded and willing to learn, particularly if you're coming from a different field.

In the early years of your career transition, I would focus on building a strong foundation in the basics of machine learning such as linear algebra, probability theory, and calculus,

In [ ]:
start_time = time.time()

# Images with text
text = """<s>[INST] Hey, can you give me an explanation of the shear stress using diagrams drawn with symbols, such as pipeline characters or dashses? [/INST] I want to understand the concept visually.
          </s>[INST] Go ahead and draw me a diagram using text symbols! [/INST]"""

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs = encodeds.to('cuda')
#model.to(device)     # to.() not supported for 8-bit. Just use model as is, since correct device has already been assigned to it

outputs = model.generate(**inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(outputs)
print(decoded[0])

print("--- %s seconds ---" % (time.time() - start_time))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Hey, can you give me an explanation of the shear stress using diagrams drawn with symbols, such as pipeline characters or dashses? [/INST] I want to understand the concept visually.
          </s>[INST] Go ahead and draw me a diagram using text symbols! [/INST] Sure! Here's a diagram using text symbols to illustrate shear stress:
```sql
          ------- -------------
         |         |
         |         |
---------|_/_\_|----------
      -   - |  -    -
         |  |      |
         |  |      |
         v  v      v
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       |       |
       v       v
          ------- -------------
```
In this diagram, the large horizontal line represents a horizontal force being applied to a rectangular block, which is divided into two sma

In [ ]:

start_time = time.time()

text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs = encodeds.to('cuda')
#model.to(device)     # to.() not supported for 8-bit. Just use model as is, since correct device has already been assigned to it

ouputs = model.generate(**inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(ouputs)
print(decoded[0])

print("--- %s seconds ---" % (time.time() - start_time))

### 1. LangChain

In [ ]:
# Define pipeline
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
# !pip install langchain-community langchain-core
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipeline)     # connect Langchain to defined pipeline

/tmp/ipykernel_33/3287174287.py:4: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipeline)     # connect Langchain to defined pipeline


In [ ]:
# Test the same prompt based on given pipeline
%%time
template = """<s>[INST] What do you think are the most important things in life? [/INST]
          I think building healthy, long-lasting relationships, a fulfilling career and exciting hobbies are vital components for a happy life!</s>
          [INST] Do you have any life advice to give someone in their early thirties seeking a career transition to machine learning, particularly in LLM's? [/INST]"""

prompt = PromptTemplate(template=template, input_variables=["",""])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":"","context":""})
response

<timed exec>:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
<timed exec>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: user 1min 20s, sys: 1.2 s, total: 1min 21s
Wall time: 1min 21s


"<s>[INST] What do you think are the most important things in life? [/INST]\n          I think building healthy, long-lasting relationships, a fulfilling career and exciting hobbies are vital components for a happy life!</s>\n          [INST] Do you have any life advice to give someone in their early thirties seeking a career transition to machine learning, particularly in LLM's? [/INST] Yes, certainly! Here are some pieces of advice for someone seeking a career transition to machine learning:\n\n1. Build a strong foundation in programming: Machine learning requires a strong foundation in programming, particularly in Python and other popular programming languages for machine learning. You should spend time learning the basics of programming, including data structures, algorithms, and linear algebra.\n2. Gain experience: The best way to learn machine learning is to gain practical experience. This can be done through internships, part-time job or freelancing, where you can work on projec

In [ ]:
# Test new prompt with different format
%%time
template = """<s>[INST] You are a helpful, respectful and honest assistant. Always provide your answer politely and exactly.
              Answer the question below from the following context:
              {context}
              {question} [/INST] </s>
            """

question_p = """How many apples does Jordan have?"""
context_p = """ Upon waking up, Jordan noticed that apples were raining from the sky. In dire need of vitamin C, Jordan swiftly went up to the roof of his house with a giant net in hand, and collected fifty-two of those heavenly fruits.
                However, he noticed while washing them that many of them became inedible from the fall and tossed out 26 of them. Unable to fight off his hunger any longer, he ate one for himself."""
prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response

/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


CPU times: user 14.4 s, sys: 557 ms, total: 15 s
Wall time: 15 s


'<s>[INST] You are a helpful, respectful and honest assistant. Always provide your answer politely and exactly.\n              Answer the question below from the following context:\n               Upon waking up, Jordan noticed that apples were raining from the sky. In dire need of vitamin C, Jordan swiftly went up to the roof of his house with a giant net in hand, and collected fifty-two of those heavenly fruits.\n                However, he noticed while washing them that many of them became inedible from the fall and tossed out 26 of them. Unable to fight off his hunger any longer, he ate one for himself.\n              How many apples does Jordan have? [/INST] </s>\n             Jordan has 25 apples left after tossing out 26 inedible fruits.'

### 1.5. RAG

In [ ]:
#!pip install chromadb #==0.3.29         #  Chroma requires sqlite3 >= 3.35.0
#!pip install pysqlite3-binary
__import__('pysqlite3')
import sys
#sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
#!pip install sentence-transformers
import pysqlite3
sys.modules['sqlite3'] = sys.modules["pysqlite3"]

import chromadb           # import chromadb after making changes to pysqlite3
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
#!pip uninstall chromadb -y
#!pip install chromadb==0.5.3
#!pip list

In [ ]:
# https://www.skysports.com/tennis/news/32833/13209759/us-open-jack-draper-books-semi-final-spot-with-straight-sets-win-over-alex-de-minaur-at-flushing-meadows
tennis_news = """Jack Draper is through to the semi-finals of the US Open, with the British No 1 making light work of 10th seed Alex de Minaur 6-3 7-5 6-2 to set up a date against good friend and world No 1 Jannik Sinner - live on Sky Sports from 8pm, Friday, September 6, 2024.
Draper's victory means he is the first British man to reach the final four at Flushing Meadows since Andy Murray won the tournament in 2012. The 22-year-old will next face top seed Sinner after the Italian defeated 2021 US Open champion Daniil Medvedev 6-2 1-6 6-1 6-4 on Wednesday night in New York.
"""

In [ ]:
from langchain.schema.document import Document
documents = [Document(page_content=tennis_news, metadata={"source": "local"})]
#######################
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
#######################
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
#######################
retriever = vectordb.as_retriever()
#######################
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

def run_my_rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)

/tmp/ipykernel_33/2853406583.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
!python3.12 -m pip list    # which chromadb

Package                                  Version
---------------------------------------- --------------------
absl-py                                  2.1.0
accelerate                               0.34.2
aiohappyeyeballs                         2.4.0
aiohttp                                  3.10.5
aiosignal                                1.3.1
annotated-types                          0.7.0
anyio                                    4.4.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.8.1
asttokens                                2.4.1
astunparse                               1.6.3
async-lru                                2.0.4
attrs                                    24.2.0
babel                                    2.16.0
backoff                                  2.2.1
bcrypt                                   4.2.0
beautifulsoup4                       

In [ ]:
query =""" Which player do you think is the underdog based on this article? """
run_my_rag(qa, query)

Query:  Which player do you think is the underdog based on this article? 



> Entering new RetrievalQA chain...

> Finished chain.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Draper's victory means he is the first British man to reach the final four at Flushing Meadows since Andy Murray won the tournament in 2012. The 22-year-old will next face top seed Sinner after the Italian defeated 2021 US Open champion Daniil Medvedev 6-2 1-6 6-1 6-4 on Wednesday night in New York.

Draper's victory means he is the first British man to reach the final four at Flushing Meadows since Andy Murray won the tournament in 2012. The 22-year-old will next face top seed Sinner after the Italian defeated 2021 US Open champion Daniil Medvedev 6-2 1-6 6-1 6-4 on Wednesday night in New York.

Jack Draper is through to the semi-finals of the US Open, with the British No 1 making lig

In [ ]:
query = """What was the date on which Sinner defeated Medvedev?"""
run_my_rag(qa, query)

Query: What was the date on which Sinner defeated Medvedev?



> Entering new RetrievalQA chain...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



> Finished chain.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Draper's victory means he is the first British man to reach the final four at Flushing Meadows since Andy Murray won the tournament in 2012. The 22-year-old will next face top seed Sinner after the Italian defeated 2021 US Open champion Daniil Medvedev 6-2 1-6 6-1 6-4 on Wednesday night in New York.

Draper's victory means he is the first British man to reach the final four at Flushing Meadows since Andy Murray won the tournament in 2012. The 22-year-old will next face top seed Sinner after the Italian defeated 2021 US Open champion Daniil Medvedev 6-2 1-6 6-1 6-4 on Wednesday night in New York.

JJack Draper is through to the semi-finals of the US Open, with the British No 1 making light work of 10th seed Alex de Minaur 6-3 7-5 6-2 to set up a date against good friend and world No 1 Jannik Sinne

In [ ]:
del pipeline
del llm
#del model_4bit
del quantization_config
del tokenizer

NameError: name 'llm' is not defined

### 2. LlamaIndex

In [ ]:
#apt-get install --upgrade python3-setuptools
#apt-get install --upgrade python3-pip
#python3.12 -m ensurepip --upgrade
#python3.12 -m pip install --upgrade six
#!pip install llama-index-readers-web
#!pip install llama-index-llms-huggingface
#!pip install llama-index-embeddings-huggingface
#!pip install llama-index-agent-openai

Traceback (most recent call last):
  File "/usr/bin/pip", line 6, in <module>
    from pkg_resources import load_entry_point
  File "/usr/lib/python3/dist-packages/pkg_resources/__init__.py", line 57, in <module>
    from pkg_resources.extern import six
ImportError: cannot import name 'six' from 'pkg_resources.extern' (/usr/lib/python3/dist-packages/pkg_resources/extern/__init__.py)


In [ ]:
# Data
from llama_index.readers.web import BeautifulSoupWebReader

url = "https://www.theverge.com/2024/9/5/24235671/anker-eufy-smart-home-ifa-permanent-outdoor-lights-e22-e10"

documents = BeautifulSoupWebReader().load_data([url])

In [ ]:
# LLM setup
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate as PromptTemplate_li
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_8bit=False,
    #bnb_4bit_compute_dtype=torch.float16,
    #bnb_4bit_quant_type="nf4",          # nf4 strikes good balance between compression and preserving model accuracy, fp4 for lower accuracy but faster initialization
    #bnb_4bit_use_double_quant=False,     # double quantization improves accuracy but adds significant overhead, increasing computation cost
)

llm_li = HuggingFaceLLM(
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    query_wrapper_prompt=PromptTemplate_li("<s>[INST] {query_str} [/INST] </s>\n"),     # formats the input query for the model
    context_window=3900,                                                                # size of the context window (number of tokens that the model can process in one go)
    max_new_tokens=256,                                                                 # maximum number of new tokens to generate (i.e., length of output text)
    #model_kwargs={"quantization_config": quantization_config},                          # additional arguments passed to the model (quantization settings)
    model_kwargs={"torch_dtype": torch.float16},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},                    # generation parameters to control randomness and sampling behavior during text generation
    device_map="auto",                                                                  # specifies where to load the model (GPU, CPU) - "auto" chooses the best available device
)


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [ ]:
from llama_index.core import Settings as Settings_li

Settings_li.llm = llm_li
Settings_li.embed_model = "local:BAAI/bge-small-en-v1.5"

In [ ]:
# Index setup using the document
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)

In [ ]:
from llama_index.core.indices import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

In [ ]:
# Logging
from llama_index.core.response.notebook_utils import display_response
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
# Basic query engine - Compact (default)
query_engine = vector_index.as_query_engine(response_mode="compact")
response = query_engine.query("How do the different Eufy products compare?")
display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:593: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:598: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
# Basic query engine - Refine
query_engine = vector_index.as_query_engine(response_mode="refine")
response = query_engine.query("How do the different Eufy products compare?")
display_response(response)

In [ ]:
# Basic query engine - Tree summarize
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")
response = query_engine.query("How do the different Eufy products compare?")
display_response(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** The different Eufy products are smart home devices that are designed to work together to create a connected home experience. The Permanent Outdoor Lights E22 are a newer, brighter version of the existing E120 model with warmer color tones. They are designed to change color based on who is walking up to your house and can be set to turn into different colors depending on the time of day or the occasion. The lights are rated IP65 for water and dust resistance and support a music sync mode and AI-curated themes.

The other RGB smart lights that Eufy is launching today include outdoor string lights, outdoor spotlights, outdoor pathway lights, and an indoor floor lamp. These lights also have the ability to change color based on the time of day or the occasion and are designed to work together with the security cameras and their local facial recognition.

Overall, the different Eufy products are designed to work together to create a connected home experience that is customizable to your needs and preferences. They are designed to be easy to use and integrate with other smart home devices, making it easy to create a smart home that is tailored to your lifestyle.

In [ ]:
# Router query engine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
# Single selector
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=False
)

response = query_engine.query("What was mentioned about the most expensive light product?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: Choice 1 is useful for searching for specific facts, which could be relevant if the most expensive light product is mentioned in a document and needs to be found..
Selecting query engine 0: Choice 1 is useful for searching for specific facts, which could be relevant if the most expensive light product is mentioned in a document and needs to be found..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** The most expensive light product mentioned in the article is the Eufy Permanent Outdoor Lights E22, which is priced at $299 for 100 feet (60 lights).

In [ ]:
# Multi selector
query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=True,
)

response = query_engine.query("What was mentioned about the most expensive light product? Summarize with other light products mentioned in the document.")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
# Sub question query engine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    verbose=True,
)

response = query_engine.query("What was mentioned about the most expensive light product? How Does it differ from the cheapest light product?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
# SQL query engine
# 11 tables with various information about music, playlists and customers
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!curl -O https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip # /app/chinook.zip
!unzip /app/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100  298k  100  298k    0     0  1208k      0 --:--:-- --:--:-- --:--:-- 1208k
Archive:  /app/chinook.zip
  inflating: chinook.db              


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////app/chinook.db")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from sqlalchemy import MetaData

metadata = MetaData()
metadata.reflect(bind=engine)
print(metadata.tables.keys())

dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])


In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
)

In [ ]:
response = query_engine.query("Tell me the names of some albums. Give me 5 of them.")

display_response(response)

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .
> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEG

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** 1. ...And Justice For All
2. 20th Century Masters - The Millennium Collection: The Best of Scorpions
3. A Copland Celebration, Vol. I
4. A Matter of Life and Death
5. A Real Dead One

In [ ]:
response = query_engine.query("How about some artists? List 5.")

display_response(response)

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .
> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEG

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Here are five artists in no particular order:

1. A Cor Do Som
2. AC/DC
3. Aaron Copland & London Symphony Orchestra
4. Aaron Goldberg
5. Academy of St. Martin in the Fields & Sir Neville Marriner

In [ ]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 5")

display_response(response)

INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEGER), Bytes (INTEGER), UnitPrice (NUMERIC(10, 2)),  and foreign keys: ['MediaTypeId'] -> media_types.['MediaTypeId'], ['GenreId'] -> genres.['GenreId'], ['AlbumId'] -> albums.['AlbumId'].

Table 'artists' has columns: ArtistId (INTEGER), Name (NVARCHAR(120)), .
> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER),  and foreign keys: ['ArtistId'] -> artists.['ArtistId'].

Table 'tracks' has columns: TrackId (INTEGER), Name (NVARCHAR(200)), AlbumId (INTEGER), MediaTypeId (INTEGER), GenreId (INTEGER), Composer (NVARCHAR(220)), Milliseconds (INTEG

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Here are five popular tracks from the band AC/DC: 

1. For Those About To Rock (We Salute You)
2. Put The Finger On You
3. Let's Get It Up
4. Inject The Venom
5. Snowballed

In [ ]:
print(response.metadata['sql_query'])

SELECT t.Name
FROM tracks t
JOIN albums a ON t.AlbumId = a.AlbumId
JOIN artists ar ON a.ArtistId = ar.ArtistId
WHERE ar.Name = 'AC/DC'
LIMIT 5;


In [ ]:
# Programs
from typing import List
from pydantic import BaseModel

from llama_index.core.program import LLMTextCompletionProgram
from llama_index.program.openai import OpenAIPydanticProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.core.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Album),
    prompt_template_str=prompt_template_str,
    llm=llm_li,
    verbose=True,
)

In [ ]:
output = program(movie_name="The Shining")

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:593: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:598: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(output)

name='The Shining Soundtrack' artist='Jack Torrance' songs=[Song(title='Redrum', length_seconds=120), Song(title='The Shining', length_seconds=180), Song(title='The Grady Hotel', length_seconds=150), Song(title='The Shining (Main Title)', length_seconds=120), Song(title='The Shining (End Title)', length_seconds=120)]


In [ ]:
# Data Agent
from llama_index.core.agent import ReActAgent
from llama_index.agent.openai import OpenAIAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm_li,
    verbose=True
)

In [ ]:
response = agent.chat("Hello!")
print(response)

/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:593: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:598: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


> Running step 534596ff-50aa-4eec-94e7-543408a599da. Step input: Hello!
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_search
Action Input: {'properties': AttributedDict([('input', AttributedDict([('title', 'Hello!'), ('type', 'string')]))]), 'required': ['input'], 'type': 'object'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Observation: 
The context information provided is about a new smart light system from Eufy that can change color based on who is at the front door. The lights are designed to work with Eufy's video doorbells and security cameras and can be set to turn into different colors depending on the person approaching. The lights are available in two sizes and can be controlled using voice commands with Alexa or Google Assistant. The lights are not compatible with the Matter smart home standard, but Eufy has its own WonderLink system for unified smart home device management. The lights are available for purchase starting at $199 for 50 feet (30 lights).
> Running step cbf90f74-4578-42e0-8b9b-d22cbe45bcce. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The new smart light system from Eufy is a color-changing light that can be controlled using voice commands with Alexa or Google Assistant. It is designed to work with Eufy's video

In [ ]:
response = agent.chat("What are your favorite songs from the given database? Why?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


> Running step 0349ea50-461d-4173-a7f0-6af4869700fb. Step input: What was mentioned about Meta? How Does it differ from how OpenAI is talked about?
Thought: (Implicit) I can answer without any more tools!
Answer: The conversation did not mention Meta, but it did mention Facebook. Facebook is a social media platform that was founded in 2004 by Mark Zuckerberg. It is one of the largest social media platforms in the world, with over 2.8 billion monthly active users as of 2021. Facebook has been involved in several controversies over the years, including the Cambridge Analytica scandal, which involved the misuse of user data by a political consulting firm.

OpenAI, on the other hand, is a research organization that was founded in 2015 by Sam Harris and Elon Musk. It is focused on developing advanced artificial intelligence and machine learning technologies. OpenAI has been involved in several projects, including the development of the GPT-2 language model and the creation of the DALL-E ima

### 2.5 LlamaIndex Alternate Method (exp.)

In [ ]:
# llama_index
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core import Settings  # ServiceContext is deprecated. Use llama_index.settings.Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding     # HuggingFaceEmbeddings from LangChain gives equivalent results
__import__('pysqlite3')
import sys
import pysqlite3
sys.modules['sqlite3'] = sys.modules["pysqlite3"]
from llama_index.vector_stores.chroma import ChromaVectorStore
from IPython.display import Markdown, display

In [ ]:
chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection('climate_report')
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'
Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


In [ ]:
# Load the data from directory
docs = SimpleDirectoryReader(input_files=["/app/IPCC_AR6_SYR_LongerReport.pdf"]).load_data()
print(f"Loaded {len(docs)} docs")

Ignoring wrong pointing object 110 0 (offset 0)
Ignoring wrong pointing object 531 0 (offset 0)
Ignoring wrong pointing object 968 0 (offset 0)
Ignoring wrong pointing object 998 0 (offset 0)
Ignoring wrong pointing object 1034 0 (offset 0)
Ignoring wrong pointing object 1221 0 (offset 0)
Ignoring wrong pointing object 1224 0 (offset 0)
Ignoring wrong pointing object 1226 0 (offset 0)
Ignoring wrong pointing object 1228 0 (offset 0)
Ignoring wrong pointing object 1230 0 (offset 0)
Ignoring wrong pointing object 1232 0 (offset 0)
Ignoring wrong pointing object 1233 0 (offset 0)
Ignoring wrong pointing object 1237 0 (offset 0)
Ignoring wrong pointing object 1354 0 (offset 0)
Ignoring wrong pointing object 1380 0 (offset 0)
Ignoring wrong pointing object 1397 0 (offset 0)
Ignoring wrong pointing object 1399 0 (offset 0)
Ignoring wrong pointing object 1402 0 (offset 0)
Ignoring wrong pointing object 1708 0 (offset 0)
Ignoring wrong pointing object 1711 0 (offset 0)
Ignoring wrong pointing 

Loaded 81 docs


In [ ]:
!huggingface-cli login

In [ ]:
from llama_index.core.prompts.prompts import SimpleInputPrompt
system_prompt = "You are a professional Q&A assistant. Your goal is to answer questions as accurately and as politely as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [ ]:
llm_li = HuggingFaceLLM(
    context_window=8000,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    tokenizer_kwargs={"max_length": 8000},
    model_kwargs={"torch_dtype": torch.float16}
)

# For the other llm_li that I didn't use
"""
llm_li = HuggingFaceLLM(
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    query_wrapper_prompt=PromptTemplate_li("<s>[INST] {query_str} [/INST] </s>\n"),     # formats the input query for the model
    context_window=3900,                                                                # size of the context window (number of tokens that the model can process in one go)
    max_new_tokens=256,                                                                 # maximum number of new tokens to generate (i.e., length of output text)
    model_kwargs={"quantization_config": quantization_config},                          # additional arguments passed to the model (quantization settings)
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},                    # generation parameters to control randomness and sampling behavior during text generation
    device_map="auto",                                                                  # specifies where to load the model (GPU, CPU) - "auto" chooses the best available device
)
"""

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

'\nllm_li = HuggingFaceLLM(\n    model_name="mistralai/Mistral-7B-Instruct-v0.1",\n    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",\n    query_wrapper_prompt=PromptTemplate_li("<s>[INST] {query_str} [/INST] </s>\n"),     # formats the input query for the model\n    context_window=3900,                                                                # size of the context window (number of tokens that the model can process in one go)\n    max_new_tokens=256,                                                                 # maximum number of new tokens to generate (i.e., length of output text)\n    model_kwargs={"quantization_config": quantization_config},                          # additional arguments passed to the model (quantization settings)\n    # tokenizer_kwargs={},\n    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},                    # generation parameters to control randomness and sampling behavior during text generation\n    device_map="auto",        

In [ ]:
embed_model =  HuggingFaceEmbedding(model_name="thenlper/gte-base")

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ServiceContext is deprecated. Use llama_index.settings.Settings - https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/
#service_context = ServiceContext.from_defaults(
#    chunk_size=1024,
#    llm=llm_li,
#    embed_model=embed_model)

Settings.chunk_size = 1024
Settings.llm = llm_li
Settings.embed_model = embed_model

In [ ]:
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)

Retrying llama_index.embeddings.openai.base.get_embeddings in 0.1270046098994 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("what does the report explain about global surface temperature ?")
display(Markdown(f"{response}"))

### 3. LangChain & LlamaIndex with CUDA

In [ ]:
# Load documents
#url = "https://www.theverge.com/2024/9/5/24235671/anker-eufy-smart-home-ifa-permanent-outdoor-lights-e22-e10"
#documents = BeautifulSoupWebReader().load_data([url])
documents = SimpleDirectoryReader(input_files=["/app/IPCC_AR6_SYR_LongerReport.pdf"]).load_data()
print(f"Loaded {len(documents)} docs")

# Create an index
index = VectorStoreIndex(documents)

Ignoring wrong pointing object 110 0 (offset 0)
Ignoring wrong pointing object 531 0 (offset 0)
Ignoring wrong pointing object 968 0 (offset 0)
Ignoring wrong pointing object 998 0 (offset 0)
Ignoring wrong pointing object 1034 0 (offset 0)
Ignoring wrong pointing object 1221 0 (offset 0)
Ignoring wrong pointing object 1224 0 (offset 0)
Ignoring wrong pointing object 1226 0 (offset 0)
Ignoring wrong pointing object 1228 0 (offset 0)
Ignoring wrong pointing object 1230 0 (offset 0)
Ignoring wrong pointing object 1232 0 (offset 0)
Ignoring wrong pointing object 1233 0 (offset 0)
Ignoring wrong pointing object 1237 0 (offset 0)
Ignoring wrong pointing object 1354 0 (offset 0)
Ignoring wrong pointing object 1380 0 (offset 0)
Ignoring wrong pointing object 1397 0 (offset 0)
Ignoring wrong pointing object 1399 0 (offset 0)
Ignoring wrong pointing object 1402 0 (offset 0)
Ignoring wrong pointing object 1708 0 (offset 0)
Ignoring wrong pointing object 1711 0 (offset 0)
Ignoring wrong pointing 

Loaded 81 docs


In [ ]:
# Set up LangChain

# Load Mistral 7B Model
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1',
                                          device_map="cuda:0",
                                          torch_dtype="auto")
model = AutoModelForCausalLM.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1',
                                             torch_dtype="auto")

# Set up LangChain prompt
pipeline = pipeline(
        "text-generation",
        device=0,
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)
llm = HuggingFacePipeline(pipeline=pipeline)
template = """<s>[INST] What do you think are the most important things in life? [/INST]
          I think building healthy, long-lasting relationships, a fulfilling career and exciting hobbies are vital components for a happy life!</s>
          [INST] Do you have any life advice to give someone in their early thirties seeking a career transition to machine learning, particularly in LLM's? [/INST]"""
#prompt = PromptTemplate(template=template, input_variables=["",""])
#llm_chain = LLMChain(prompt=prompt, llm=llm)
inputs = tokenizer(text, return_token_type_ids=False, return_tensors="pt").to(device)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Retrieve relevant documents for: {query}"
)

# Create the LLM Chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: The following `model_kwargs` are not used by the model: ['device', 'model', 'device_map'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
# Integrate LlamaIndex with LangChain
def query_index(query):
    response = index.query(query)  # Query LlamaIndex
    return response.response  # Return the document response

user_query = "What is the latest on e-commerce trends?"
documents = query_index(user_query)  # Retrieve docs via LlamaIndex

# Feed documents to LangChain for further processing
llm_chain.run(query=user_query + " " + documents)

### 3.5. RAG using LangChain & LlamaIndex with CUDA & TensorRT (CUDA-X)

In [ ]:
##### Need test


from langchain import OpenAI, RetrievalQA
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext

# Using the LlamaIndex to perform document retrieval
def query_index(query):
    retriever = index.as_retriever()
    return retriever.retrieve(query)

# Function to generate response using the retrieved documents
def generate_with_rag(query):
    # Retrieve relevant documents
    docs = query_index(query)

    # Concatenate the document text for LLM input
    doc_texts = ' '.join([doc.text for doc in docs])
    input_text = query + " " + doc_texts

    # Generate response using LLM (using GPU with TensorRT optimization)
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the RAG workflow
query = "Explain the benefits of using TensorRT with LLMs."
print(generate_with_rag(query))

ImportError: cannot import name 'RetrievalQA' from 'langchain' (/usr/local/lib/python3.12/dist-packages/langchain/__init__.py)

In [ ]:
# Convert PyTorch model to TensorRT (Optimization Step)
import tensorrt as trt
from torch2trt import torch2trt

# Assuming 'model' is already loaded from HuggingFace
model = model.to("cuda").eval()

# Input data format for TensorRT conversion (adjust based on actual input shapes)
inputs = torch.ones((1, 1), dtype=torch.long).to("cuda")

# Convert the model to TensorRT engine
model_trt = torch2trt(model, [inputs], max_batch_size=1)

# Test inference using the optimized TensorRT model
def generate_with_tensorrt_rag(query):
    # Retrieve relevant documents using LlamaIndex
    docs = query_index(query)
    doc_texts = ' '.join([doc.text for doc in docs])
    input_text = query + " " + doc_texts

    # Tokenize input for the LLM
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

    # Generate with TensorRT-optimized model
    with torch.no_grad():
        outputs = model_trt.generate(**inputs, max_length=512)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the optimized TensorRT model
query = "What is the role of TensorRT in optimizing LLM inference?"
print(generate_with_tensorrt_rag(query))


### Notes: Using cuDF (RAPIDS)

In [ ]:
!python3.12 -m pip install cudf

In [ ]:
##### Need test


import cudf
from cudf.core.column.string import StringColumn
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype="auto")

# Create large dataset using cuDF
text_data = ["This is a large dataset."] * 1000000
df = cudf.DataFrame({'text': text_data})

# Preprocessing: tokenize text using RAPIDS and CUDA
tokenized_text = df['text'].applymap(lambda x: tokenizer(x)['input_ids'])

# Move processed data to GPU and run inference
inputs = tokenized_text.to_gpu_array()
# Here you would pass the `inputs` to your Mistral model for processing.

### Prompt Format
```
<s>[INST] <user_prompt> [/INST]

<assistant_response> </s>

[INST] <user_prompt>[/INST]
```

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
def generate(input_text, system_prompt="",max_length=512):
    prompt = f"""<s>[INST]{input_text}[/INST]"""
    inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    wrapped_text = wrap_text(text)
    print(wrapped_text)

## CodeGen

In [ ]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

In [ ]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

## Instruction Answering

In [ ]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         max_length=128)

In [ ]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

In [ ]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

In [ ]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

In [ ]:
%%time
generate('What is the capital of England?',
         max_length=256)

In [ ]:
%%time
generate('Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.',
         max_length=256)

In [ ]:
generate('Write a story about a Koala playing pool and beating all the camelids.',
         max_length=512)

## Chat

In [ ]:
generate("""Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:""",
         max_length=128)

## GSM8K

In [ ]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

In [ ]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=128)

In [ ]:
generate("A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?",
         max_length=256)

In [ ]:
# Reference:
# langchain
# https://colab.research.google.com/drive/1quYAZ5Od4PtwfhPjXmMI1dwyg1eLTN_h
# https://colab.research.google.com/drive/1TWOC6waBUk7SY3T2hQi05skrTaZ7V13W?source=post_page-----129fa5e9a04d--------------------------------#scrollTo=-Dwb8zbHjdbN
# GPU acceleration
# https://lolevsky.medium.com/running-locally-llama-and-langchain-accelerated-by-gpu-a52a2fd72d79
# llama-index
# https://colab.research.google.com/drive/1-f5v48TnX5rGdaMdWTr8XsjTGrWZ6Q7Y?usp=sharing#scrollTo=iiS0z1UxWgyt
# https://github.com/mickymultani/RAG-ChromaDB-Mistral7B/blob/main/RAG_Chromadb_mistral7b.ipynb